In [ ]:
import openai
openai.api_key = '###'

In [6]:
import os
# os.getcwd()
os.chdir('/home/mmilkov2/projects/VK/!for_OSF/scripts_retrieving_preprocessing_annotating/')
# os.getcwd()

In [14]:
text_guidline_file='../instructions/Instruction_for_ChatGPT.txt'
with open(text_guidline_file) as f:
    text_lines = f.read().splitlines()
text_instruction=' '. join(text_lines)
text_instruction

'Перед Вами пост из социальной сети ВКонтакте. Ваша задача – определить, есть ли выражение ценности в посте.  Вы будете отвечать на вопрос: Отражает ли пост важность какого-либо объекта,  явления или процесса?  Выберите одну из четырех категрий: Да отражает; Нет, не отражает; Неясно; Спам. Ценности – это убеждение человека в важности какого-либо объекта, явления или качества. Выберите Да, отражает, если текст представляет собой: Важность романтических отношений, любви, верности, дружбы, преданности близким (в том числе, любовь к детям);  Важность равенства всех людей или сбережение природы;  Желание стабильности и порядка, следования правилам, важности безопасности, необходимости беречь свое здоровье;  Важность быть независимым, желание пробовать новые вещи, рисковать;  Важность достижения целей, получения заслуженных наград;  Важность просто наслаждения жизнью.  Пожалуйста, имейте ввиду, что в текстах могут встречаться и другие виды ценностей - учитывайте свое собственное понимание!  

In [ ]:
post='Пост: '
answer='Ответ: '
prompt_len=num_tokens_from_messages(messages = [{"role": "user", "content": text_instruction+'\n'+post+'\n'+answer}])

def AreThereValuesInThePost (instruction, text, prompt_len):
     
    #calculate length of the text. if > than max_length, then - cut
    max_len=4097
    text_len=num_tokens_from_messages(messages = [{"role": "user", "content": text}])

    real_len=text_len+prompt_len+16
    if real_len>max_len:
        cutted_len=(real_len-max_len)*3
        text=text[:-cutted_len]
    
    post='Пост: '
    answer='Ответ: '
    
    prompt=text_instruction+'\n'+post+text+'\n'+answer
    
    messages = [{"role": "user", "content": prompt}]
    
    response = openai.ChatCompletion.create(
                            model='gpt-3.5-turbo', 
                            messages=messages,
                            max_tokens=16, 
                            stop='.',
                            n=1,
                            temperature=0,
                        )

    output = response['choices'][0]['message']['content'].strip()
    return output

# Load Training dataset

In [89]:
import pandas as pd

labeling_folder='../data/'
file_random=pd.read_csv(labeling_folder+'!toloka_random.csv', sep="|", encoding ='utf-8')
file_pre_selected=pd.read_csv(labeling_folder+'!toloka_pre_selected.csv', sep="|", encoding ='utf-8')
file_1post1id=pd.read_csv(labeling_folder+'!toloka_1post1id.csv', sep="|", encoding ='utf-8')
df=pd.concat([file_random, file_pre_selected, file_1post1id])
df=df.rename(columns={'INPUT:headline':'text', 'label_1':'label_c1', 'label_2':'label_c2', 'label_3':'label_c3','result': 'label_crowd'})
print (df.shape[0])
df.head()


5035


,text,label_c1,label_c2,label_c3,label_crowd,percentage
0,"Думаете, что умеете пользоваться фотошопом?...",3.0,0.0,0.0,0.0,67.0
1,...Самое страшное - это когда ты стоишь под х...,1.0,1.0,1.0,1.0,100.0
2,Друзья мои! Поддержим дочку моей подруги! Про...,0.0,1.0,0.0,0.0,67.0
3,"Мой новый дневник, читаем, коментим :)",0.0,0.0,0.0,0.0,100.0
4,РУССКИЙ КРЫМ - МИФ для быдла! (о чем молчат ...,0.0,0.0,3.0,0.0,67.0


script for annotating train dataset by ChatGPT. If need trial dataset, please, load text items from 
trial804_experts_crowd_gpt.csv

In [ ]:
import time
from tqdm.notebook import tqdm

retries = 300

for i in range (0,3):
    col_name = 'label' + str(i+1)
    df[col_name] = pd.Series(dtype='str')

    it = 0        
    for index, row in tqdm(df.iterrows()):
        while retries > 0:
            try:
                label=AreThereValuesInThePost(text_instruction, row.text, prompt_len)
                df_all1.at[index, col_name] = label
                print (row.text[:50])
                print (label)
                print ("")
                i+=1
                break  # Exit the retry loop if the request is successful
            except openai.error.OpenAIError as e:
                print(f"Error: {e}")
                retries -= 1
                time.sleep(20)  # Wait for 20 seconds before retrying
                    

In [30]:
# In case if ChatGPT return not only labels, but labels with text

def Convert(df, cols, resulted_col_name):
    
    for col in cols:
        df[col]=df[col].astype('str')
        df[col] = df[col].str.split('(', 1).str[0]
        df[col]=df[col].str.replace('Не отражает', 'Нет, не отражает')
        df[col] = df[col].apply(lambda x: x.strip())
        df[col]=df[col].str.replace('Да, отражает', '1')
        df[col]=df[col].str.replace('Отражает ценность любви и преданности б', '1')
        df[col]=df[col].str.replace('Отражает ценность романтических от', '1')
        df[col]=df[col].str.replace('Нет, не отражает', '0')
        df[col]=df[col].str.replace('Неясно', '2')
        df[col]=df[col].str.replace('Спам', '3')
        df[col]=df[col].astype(float).astype(int)
    for i, row in df.iterrows():
        row_list=row[cols].to_list()
        common_value=LabelPercentage(row_list)
        df.at[i, resulted_col_name]=common_value
    return df


In [31]:
# Calculate the percentage of identical labels
from collections import Counter
def LabelPercentage(row_list):

    c = Counter(row_list)
    most_common=c.most_common(1)
    common_value=most_common[0][0]
    perc=round((most_common[0][1]/len(row_list))*100, 0)
    if perc==33.0:
        perc=0
        common_value='Nan'
    return common_value

In [102]:
df=Convert(df, ['label1', 'label2', 'label3'], 'gpt_result')

# Count number of tokens for ChatGPT

In [ ]:
import tiktoken


def num_tokens_from_messages(messages, model="gpt-3.5-turbo"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":
        print("Warning: gpt-3.5-turbo may change over time. Returning num tokens assuming gpt-3.5-turbo-0301.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301")
    elif model == "gpt-4":
        print("Warning: gpt-4 may change over time. Returning num tokens assuming gpt-4-0314.")
        return num_tokens_from_messages(messages, model="gpt-4-0314")
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif model == "gpt-4-0314":
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [ ]:
messages = [{"role": "user", "content": text}]
num_tokens_from_messages(messages)